### [Multiple-ML Time Series Forecasting (TSF) across Industries](https://python.plainenglish.io/using-skforecast-for-multiple-ml-time-series-forecasting-tsf-across-industries-3-fe07f82b797b)

> Skforecast Recursive Multi-Step TSF with LightGBM Model Tuning & Feature Dominance Analysis

The ultimate goal  is to unleash the power of Skforecast recursive multi-step TSF [1–8], including HPO, backtesting, feature engineering, probabilistic forecasting, and ML interpretation modules in Python.

**_LightGBM_** is a fast, distributed, high-performance gradient boosting framework that uses a tree-based ML algorithm. It also supports GPU learning and is thus widely used for data science business applications.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.environ['SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL'] = 'True'

In [ ]:
!pip install -q numpy pandas scikit-learn matplotlib seaborn statsmodels PyAstronomy
!pip install -q skforecast plotly xgboost catboost lightgbm sklearn shap feature-engine

In [ ]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd
from skforecast.datasets import fetch_dataset
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from feature_engine.timeseries.forecasting import WindowFeatures

# Plots
# ==============================================================================
import matplotlib.pyplot as plt

from skforecast.plot import plot_residuals
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
plt.style.use('seaborn-v0_8-darkgrid')

# Modelling and Forecasting
# ==============================================================================
import xgboost
import lightgbm
import catboost
import sklearn
import shap
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

import skforecast
from skforecast.recursive import ForecasterEquivalentDate
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, OneStepAheadFold
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.feature_selection import select_features
from skforecast.preprocessing import RollingFeatures

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('once')

color = '\033[1m\033[38;5;208m' 
print(f"{color}Version skforecast: {skforecast.__version__}")
print(f"{color}Version scikit-learn: {sklearn.__version__}")
print(f"{color}Version lightgbm: {lightgbm.__version__}")
print(f"{color}Version xgboost: {xgboost.__version__}")
print(f"{color}Version catboost: {catboost.__version__}")
print(f"{color}Version pandas: {pd.__version__}")
print(f"{color}Version numpy: {np.__version__}")

In [ ]:
data = fetch_dataset(name='bike_sharing', raw=True)
data.tail()

In [ ]:
# Preprocessing data (setting index and frequency)
# ==============================================================================
data = data[['date_time', 'users', 'holiday', 'weather', 'temp', 'atemp', 'hum', 'windspeed']]
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('date_time')
data = data.asfreq('h')
data = data.sort_index()
data.head()

In [ ]:
# Split train-validation-test
# ==============================================================================
end_train = '2012-03-30 23:59:00'
end_validation = '2012-07-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validation : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

In [ ]:
# Interactive plot of time series
# ==============================================================================
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['users'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=data_val.index, y=data_val['users'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['users'], mode='lines', name='Test'))
fig.update_layout(
    title  = 'Number of users',
    xaxis_title="Time",
    yaxis_title="Users",
    legend_title="Partition:",
    width=950,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1, xanchor="left", x=0.001)
)

In [ ]:
# Create forecaster
# ==============================================================================
window_features = RollingFeatures(stats=["mean"], window_sizes=24 * 3)
forecaster = ForecasterRecursive(
                regressor       = LGBMRegressor(random_state=123, verbose=-1),
                lags            = 24,
                window_features = window_features
             )

# Train forecaster
# ==============================================================================
forecaster.fit(y=data.loc[:end_validation, 'users'])
forecaster

In [ ]:
# Predict
# ==============================================================================
forecaster.predict(steps=10)

In [ ]:
# Backtest model on test data
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 36,
        initial_train_size = len(data[:end_validation]),
        refit              = False,
)
metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = data['users'],
                          cv            = cv,
                          metric        = 'mean_absolute_error',
                          n_jobs        = 'auto',
                          verbose       = False,  # Change to True to see more information
                          show_progress = True
                      )
predictions.head()

In [ ]:
predictions.tail()

In [ ]:
# Backtesting error
# ==============================================================================
metric

In [ ]:
date_range = data_test['users'].loc['2012-08-01 00:00:00':'2012-12-31 23:00:00']

fig, ax = plt.subplots(figsize=(12, 6))
date_range.plot(ax=ax, label='Test')
predictions.plot(ax=ax, label='Predictions', linestyle='-')
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.legend();

In [ ]:
plt.scatter(date_range,predictions)
plt.plot(date_range,date_range,c='r')
plt.xlabel('Test')
plt.ylabel('Prediction')

In [ ]:
from sklearn.metrics import r2_score
r2_score(date_range,predictions)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(date_range,predictions)

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(date_range,predictions)

In [ ]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(date_range,predictions)

##### Baseline Hyperparameter Optimization with Backtesting

In [ ]:
# Hyperparameters search
# ==============================================================================
# Lags grid
lags_grid = [36,48, 72, [1, 3, 6, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

# Folds training and validation
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data_train),
        refit              = False,
    )

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster    = forecaster,
    y             = data.loc[:end_validation, 'users'], # Test data not used
    cv            = cv,
    search_space  = search_space,
    metric        = 'mean_absolute_error',
    n_trials      = 20, # Increase this value for a more exhaustive search
    random_state  = 123,
    return_best   = True,
    n_jobs        = 'auto',
    verbose       = False,
    show_progress = True
)

In [ ]:
n_trials = 20
results_search.info()

In [ ]:
x = np.linspace(0, n_trials-1, n_trials)
plt.plot(x,results_search['learning_rate'])
plt.xticks(np.arange(0, 19, step=1))
plt.title('Learning Rate')
plt.xlabel('Trial No')

In [ ]:
x = np.linspace(0, n_trials-1, n_trials)
plt.plot(x,results_search['n_estimators'])
plt.xticks(np.arange(0, 19, step=1))
plt.title('N Estimators')
plt.xlabel('Trial No')

In [ ]:
x = np.linspace(0, n_trials-1, n_trials)
plt.plot(x,results_search['max_depth'])
plt.xticks(np.arange(0, 19, step=1))
plt.title('Max Depth')
plt.xlabel('Trial No')

In [ ]:
x = np.linspace(0, n_trials-1, n_trials)
plt.plot(x,results_search['reg_lambda'])
plt.xticks(np.arange(0, 19, step=1))
plt.title('Reg Lambda')
plt.xlabel('Trial No')

In [ ]:
x = np.linspace(0, n_trials-1, n_trials)
plt.plot(x,results_search['reg_alpha'])
plt.xticks(np.arange(0, 19, step=1))
plt.title('Reg Alpha')
plt.xlabel('Trial No')

In [ ]:
# Best model
# ==============================================================================
forecaster

In [ ]:
# Backtest final model on test data
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data[:end_validation]),
        refit              = False,
     )
metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = data['users'],
                          cv            = cv,
                          metric        = 'mean_absolute_error',
                          n_jobs        = 'auto',
                          verbose       = False,
                          show_progress = True
                      )
display(metric)

In [ ]:
predictions.head()

In [ ]:
# Plot predictions vs real value
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['users'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="Users",
    width=950,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show()

In [ ]:
plt.scatter(date_range,predictions)
plt.plot(date_range,date_range,c='r')
plt.xlabel('Test')
plt.ylabel('Prediction')

In [ ]:
r2_score(date_range,predictions)

In [ ]:
mean_absolute_percentage_error(date_range,predictions)

In [ ]:
explained_variance_score(date_range,predictions)

In [ ]:
root_mean_squared_error(date_range,predictions)

##### Training LGBMRegressor with Exogeneous Variables

In [ ]:
# Calendar features
# ==============================================================================
features_to_extract = [
    'month',
    'week',
    'day_of_week',
    'hour'
]
calendar_transformer = DatetimeFeatures(
    variables='index',
    features_to_extract=features_to_extract,
    drop_original=True,
)
calendar_features = calendar_transformer.fit_transform(data)[features_to_extract]

In [ ]:
# Sunlight features
# ==============================================================================
from astronomy import LocationInfo, sun
location = LocationInfo(
    name      = 'Washington DC',
    region    = 'USA',
    timezone  = 'US/Eastern',
    latitude  = 40.516666666666666,
    longitude = -77.03333333333333
)
sunrise_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour
    for date in data.index
]
sunset_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour
    for date in data.index
]
sun_light_features = pd.DataFrame({
                        'sunrise_hour': sunrise_hour,
                        'sunset_hour': sunset_hour}, 
                        index = data.index
                     )
sun_light_features['daylight_hours'] = (
    sun_light_features['sunset_hour'] - sun_light_features['sunrise_hour']
)
sun_light_features["is_daylight"] = np.where(
    (data.index.hour >= sun_light_features["sunrise_hour"])
    & (data.index.hour < sun_light_features["sunset_hour"]),
    1,
    0,
)

In [ ]:
# Holiday features
# ==============================================================================
holiday_features = data[['holiday']].astype(int)
holiday_features['holiday_previous_day'] = holiday_features['holiday'].shift(24)
holiday_features['holiday_next_day'] = holiday_features['holiday'].shift(-24)

In [ ]:
# Rolling windows of temperature
# ==============================================================================
wf_transformer = WindowFeatures(
    variables   = ["temp"],
    window      = ["1D", "7D"],
    functions   = ["mean", "max", "min"],
    freq        = "h",
)
temp_features = wf_transformer.fit_transform(data[['temp']])

In [ ]:
# Merge all exogenous variables
# ==============================================================================
assert all(calendar_features.index == sun_light_features.index)
assert all(calendar_features.index == temp_features.index)
assert all(calendar_features.index == holiday_features.index)
df_exogenous_features = pd.concat([
    calendar_features,
    sun_light_features,
    temp_features,
    holiday_features
], axis=1)

# Due to the creation of moving averages, there are missing values at the beginning
# of the series. And due to holiday_next_day there are missing values at the end.
df_exogenous_features = df_exogenous_features.iloc[7 * 24:, :]
df_exogenous_features = df_exogenous_features.iloc[:-24, :]
df_exogenous_features.head(5)

In [ ]:
df_exogenous_features.info()

In [ ]:
features_to_encode = [
    "month",
    "week",
    "day_of_week",
    "hour",
    "sunrise_hour",
    "sunset_hour",
]
max_values = {
    "month": 12,
    "week": 52,
    "day_of_week": 6,
    "hour": 23,
    "sunrise_hour": 23,
    "sunset_hour": 23,
}
cyclical_encoder = CyclicalFeatures(
    variables     = features_to_encode,
    max_values    = max_values,
    drop_original = False
)

df_exogenous_features = cyclical_encoder.fit_transform(df_exogenous_features)
#df_exogenous_features.tail(3)

In [ ]:
transformer_poly = PolynomialFeatures(
                        degree           = 2,
                        interaction_only = True,
                        include_bias     = False
                    ).set_output(transform="pandas")
poly_cols = [
    'month_sin', 
    'month_cos',
    'week_sin',
    'week_cos',
    'day_of_week_sin',
    'day_of_week_cos',
    'hour_sin',
    'hour_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'daylight_hours',
    'is_daylight',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_window_1D_mean',
    'temp_window_1D_min',
    'temp_window_1D_max',
    'temp_window_7D_mean',
    'temp_window_7D_min',
    'temp_window_7D_max',
    'temp',
    'holiday'
]
poly_features = transformer_poly.fit_transform(df_exogenous_features[poly_cols])
poly_features = poly_features.drop(columns=poly_cols)
poly_features.columns = [f"poly_{col}" for col in poly_features.columns]
poly_features.columns = poly_features.columns.str.replace(" ", "__")
assert all(poly_features.index == df_exogenous_features.index)
df_exogenous_features = pd.concat([df_exogenous_features, poly_features], axis=1)
df_exogenous_features.head(3)

In [ ]:
data["weather"] = data["weather"].astype("category")

In [ ]:
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        make_column_selector(dtype_include=['category', 'object']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")  

In [ ]:
forecaster = ForecasterRecursive(
                regressor        = LGBMRegressor(random_state=123, verbose=-1),
                lags             = 72,
                window_features  = window_features,
                transformer_exog = one_hot_encoder
             )

In [ ]:
exog_features = ['weather']         
X_train, y_train = forecaster.create_train_X_y(
                        y    = data.loc[:end_validation, 'users'],
                        exog = data.loc[:end_validation, exog_features]
                   )

In [ ]:
X_train.head(3)

In [ ]:
X_train.shape

In [ ]:
X_train.info()

In [ ]:
pipeline_categorical = make_pipeline(
    OrdinalEncoder(
        dtype=int,
        handle_unknown="use_encoded_value",
        unknown_value=-1,
        encoded_missing_value=-1
    ),
    FunctionTransformer(
        func=lambda x: x.astype('category'),
        feature_names_out= 'one-to-one'
    )
)

In [ ]:
transformer_exog = make_column_transformer(
    (
        pipeline_categorical,
        make_column_selector(dtype_include=['category', 'object']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

In [ ]:
forecaster = ForecasterRecursive(
                regressor        = LGBMRegressor(random_state=123, verbose=-1),
                lags             = 72,
                window_features  = window_features,
                transformer_exog = transformer_exog,
                fit_kwargs       = {"categorical_feature": "auto"}
             )

In [ ]:
exog_features = []
# Columns that ends with _sin or _cos are selected
exog_features.extend(df_exogenous_features.filter(regex='_sin$|_cos$').columns.tolist())
# columns that start with temp_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^temp_.*').columns.tolist())
# Columns that start with holiday_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^holiday_.*').columns.tolist())
exog_features.extend(['temp', 'holiday', 'weather'])

df_exogenous_features = df_exogenous_features.filter(exog_features, axis=1)
# Merge target and exogenous variables in the same dataframe
# ==============================================================================
data = data[['users', 'weather']].merge(
            df_exogenous_features,
            left_index=True,
            right_index=True,
            how='inner' # To use only dates for which we have all the variables
        )
data = data.astype({col: np.float32 for col in data.select_dtypes("number").columns})
data_train = data.loc[: end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()

In [ ]:
forecaster = ForecasterRecursive(
                regressor        = LGBMRegressor(random_state=123, verbose=-1),
                lags             = 72,
                window_features  = window_features,
                transformer_exog = transformer_exog,
                fit_kwargs       = {"categorical_feature": "auto"}
             )

# Lags grid
lags_grid = [36,48, 72, [1, 3, 6, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

# Folds
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data_train),
        refit              = False
     )

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster    = forecaster,
    y             = data.loc[:end_validation, 'users'],
    exog          = data.loc[:end_validation, exog_features],
    cv            = cv,
    search_space  = search_space,
    metric        = 'mean_absolute_error',
    n_trials      = 20,
    random_state  = 123,
    return_best   = True,
    n_jobs        = 'auto',
    verbose       = False,
    show_progress = True
)
best_params = results_search['params'].iat[0]
best_params = best_params | {'random_state': 123, 'verbose': -1}

In [ ]:
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data[:end_validation]),
        refit              = False,
     )

metric, predictions = backtesting_forecaster(
                        forecaster    = forecaster,
                        y             = data['users'],
                        exog          = data[exog_features],
                        cv            = cv,
                        metric        = 'mean_absolute_error',
                        n_jobs        = 'auto',
                        verbose       = False,
                        show_progress = True
                     )
metric

In [ ]:
# Plot predictions vs real value
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['users'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="Users",
    width=950,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show()

In [ ]:
plt.scatter(data_test['users'],predictions['pred'])
plt.plot(data_test['users'],data_test['users'],c='r')
plt.xlabel('Test')
plt.ylabel('Prediction')

In [ ]:
r2_score(data_test['users'],predictions['pred'])

In [ ]:
mean_absolute_percentage_error(data_test['users'],predictions['pred'])

In [ ]:
explained_variance_score(data_test['users'],predictions['pred'])

In [ ]:
root_mean_squared_error(data_test['users'],predictions['pred'])

##### Out-of-Sample Predictions with 90% Confidence Interval

In [ ]:
# Create forecaster

regressor = LGBMRegressor(
   n_estimators=600, 
    max_depth=4, 
    min_data_in_leaf=27,
    learning_rate=0.09924212335795864, 
    feature_fraction=0.7, 
    max_bin=225, 
    reg_alpha=0.5, 
    reg_lambda=0.2,
    random_state = 123,
    verbose      = -1
)
forecaster = ForecasterRecursive(
    regressor        = regressor,
    lags             = [36,48, 72, 1, 3, 6, 23, 24, 25, 167, 168, 169],
    window_features  = window_features,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"}
)

# Recursive feature elimination with cross-validation
# ==============================================================================
selector = RFECV(
    estimator = regressor,
    step      = 1,
    cv        = 3,
    n_jobs    = -1
)
selected_lags, selected_window_features, selected_exog = select_features(
    forecaster      = forecaster,
    selector        = selector,
    y               = data_train['users'],  
    exog            = data_train[exog_features],
    select_only     = None,
    force_inclusion = None,
    subsample       = 0.5,
    random_state    = 123,
    verbose         = True,
)

In [ ]:
forecaster = ForecasterRecursive(
    regressor        = LGBMRegressor(**best_params),
    lags             = selected_lags,
    window_features  = window_features,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"}
)

In [ ]:
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data[:end_validation]),
        refit              = False,
)
metric_lgbm, predictions = backtesting_forecaster(
    forecaster    = forecaster,
    y             = data['users'],
    exog          = data[selected_exog],
    cv            = cv,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
    show_progress = True
)
metric_lgbm

In [ ]:
exog_features = selected_exog

In [ ]:
forecaster = ForecasterRecursive(
    regressor        = LGBMRegressor(**best_params),
    lags             = 72,
    window_features  = window_features,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"},
    binner_kwargs    = {"n_bins": 20}
)
forecaster.fit(
    y    = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)
# Predict intervals
# ==============================================================================
# Since the model has been trained with exogenous variables, they must be provided
# for the prediction.
predictions = forecaster.predict_interval(
    exog     = data.loc[end_train:, exog_features],
    steps    = 6,
    interval = [5, 95], 
)
predictions.head()

In [ ]:
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data.loc[:end_train]),
        refit              = False,
)
_, predictions_val = backtesting_forecaster(
    forecaster    = forecaster,
    y             = data.loc[:end_validation, 'users'],
    exog          = data.loc[:end_validation, exog_features],
    cv            = cv,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
    show_progress = True
)

In [ ]:
residuals = data.loc[predictions_val.index, 'users'] - predictions_val['pred']
print(pd.Series(np.where(residuals < 0, 'negative', 'positive')).value_counts())
plt.rcParams.update({'font.size': 8})
_ = plot_residuals(
        y_true = data.loc[predictions_val.index, 'users'],
        y_pred = predictions_val['pred'],
        figsize=(7, 4)
    )

In [ ]:
forecaster.set_out_sample_residuals(
    y_true = data.loc[predictions_val.index, 'users'],
    y_pred = predictions_val['pred']
)

In [ ]:
end_test = '2012-12-30 23:00:00'
cv = TimeSeriesFold(
        steps              = 6,
        initial_train_size = len(data.loc[:end_validation]),
        refit              = False,
    )
metric, predictions = backtesting_forecaster(
   forecaster              = forecaster,
   y                       = data.loc[:end_test, 'users'],
   exog                    = data.loc[:end_test, exog_features],
   cv                      = cv,
   metric                  = 'mean_absolute_error',
   interval                = [5, 95], # 90% prediction interval
   n_boot                  = 150,
   use_in_sample_residuals = False,  # Use out-sample residuals
   use_binned_residuals    = True,   # Use residuals conditioned on predicted values
   n_jobs                  = 'auto',
   verbose                 = False,
   show_progress           = True
)
predictions.head(5)

In [ ]:
fig = go.Figure([
    go.Scatter(name='Prediction', x=predictions.index, y=predictions['pred'], mode='lines'),
    go.Scatter(
        name='Real value', x=data_test.loc[:end_test].index, y=data_test.loc[:end_test, 'users'], mode='lines',
    ),
    go.Scatter(
        name='Upper Bound', x=predictions.index, y=predictions['upper_bound'], mode='lines',
        marker=dict(color="#444"), line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        name='Lower Bound', x=predictions.index, y=predictions['lower_bound'], marker=dict(color="#444"),
        line=dict(width=0), mode='lines', fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty', showlegend=False
    )
])
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="users",
    width=950,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    hovermode="x",
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001),
    # Initial zoom on x axis betwee 1 oct to 10 oct
    xaxis=dict(range=['2012-10-01', '2012-10-15'])
)
fig.show()

In [ ]:
coverage = np.mean(
    np.logical_and(
        data.loc[end_validation:end_test, 'users'] >= predictions["lower_bound"],
        data.loc[end_validation:end_test, 'users'] <= predictions["upper_bound"]
    )
)
area = (predictions['upper_bound'] - predictions['lower_bound']).sum()
print(f"Total area of the interval: {round(area, 2)}")
print(f"Predicted interval coverage: {round(100 * coverage, 2)} %")

In [ ]:
predictions = forecaster.predict(steps=24*3, exog=data_test[exog_features])
predictions

In [ ]:
from datetime import datetime

date_range = data_test['users'].loc['2012-08-01':'2012-08-03']

fig, ax = plt.subplots(figsize=(6, 2.5))
date_range.plot(ax=ax, label='Test')
predictions.plot(ax=ax, label='Predictions', linestyle='--')
ax.set_xlabel(None)
ax.legend();

##### Feature Importance Analysis

In [ ]:
forecaster = ForecasterRecursive(
    regressor        = LGBMRegressor(**best_params),
    lags             = [36,48, 72, 1, 3, 6, 23, 24, 25, 167, 168, 169],
    window_features  = window_features,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"}
)
forecaster.fit(
    y    = data.loc[:end_validation, 'users'],
    exog = data.loc[:end_validation, exog_features]
)

In [ ]:
importance = forecaster.get_feature_importances()
importance.head(10)

In [ ]:
imp15=importance.head(10)
#https://stackoverflow.com/questions/59559682/how-to-change-pandas-dataframe-plot-fontsize-of-xlabel
ax=imp15.plot.bar(x='feature', y='importance', rot=0,figsize=(18,6),fontsize=16)
ax.legend(loc='upper right',fontsize=16)
ax.set_xlabel('Feature',fontdict={'fontsize':16})

##### Using SHAP Tree Explainer

In [ ]:
explainer = shap.TreeExplainer(forecaster.regressor)

# Sample 50% of the data to speed up the calculation
rng = np.random.default_rng(seed=123)
sample = rng.choice(X_train.index, size=int(len(X_train)*0.5), replace=False)
X_train_sample = X_train.loc[sample, :]
shap_values = explainer.shap_values(X_train_sample)

# Shap summary plot (top 10)
# ==============================================================================
shap.initjs()
shap.summary_plot(shap_values, X_train_sample, max_display=10, show=False)
fig, ax = plt.gcf(), plt.gca()
ax.set_title("SHAP Summary plot")
ax.tick_params(labelsize=14)
fig.set_size_inches(10, 8)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0, :], X_train_sample.iloc[0, :])

In [ ]:
# https://cienciadedatos.net/documentos/py57-interpretable-forecasting-models.html
shap.summary_plot(shap_values, X_train, plot_type="bar", plot_size=(12,8))

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[:300, :], X_train_sample.iloc[:300, :])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
shap.dependence_plot("lag_1", shap_values, X_train_sample, ax=ax)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

#

fig, ax = plt.subplots(figsize=(8, 3))
pd.plots = PartialDependenceDisplay.from_estimator(
    estimator = forecaster.regressor,
    X         = X_train,
    features  = ["lag_168", "lag_1"],
    kind      = 'both',
    ax        = ax,
)
ax.set_title("Partial Dependence Plot")
fig.tight_layout();